## Week 2: Introduction to Supervised Learning 1
### Goal of this notebook:
The goal of this notebook is to apply K-Nearest Neighbour (KNN) and Bagging Algorithm on the iris dataset. 

In [9]:
from sklearn import datasets
iris_data, iris_labels = datasets.load_iris(return_X_y=True, as_frame=False)
print("The dimensions of the Iris feature matrix", iris_data.shape)
print(iris_data[:5])

The dimensions of the Iris feature matrix (150, 4)
[[5.1 3.5 1.4 0.2]
 [4.9 3.  1.4 0.2]
 [4.7 3.2 1.3 0.2]
 [4.6 3.1 1.5 0.2]
 [5.  3.6 1.4 0.2]]


## Explore the dataset
Right, now let's explore the dataset and see the labels and features.

* Read about the Iris dataset here: https://scikit-learn.org/stable/datasets/toy_dataset.html
* What type of labels does it have (real continuous or categorical)? What kind of machine learning task is this type of label suited to, i.e. classification or regression?
  * Based on documentation and after inspecting the dataset, it has categorical labels, so classification is more appropriate for this dataset.
* What is the feature dimensionality of the dataset, i.e. the number of features?
  * it has 4 features: Sepal length, Sepal width, Petal length, Petal width
* How many data instances are there? What is the distribution of instances across classes?
  * 150 data instances, distributed equally



---


* Select one of the features. What association does the selected feature have with the iris classes, with respect to differentiating between them (Hint - use a search engine to read about Iris Setosa, Iris Versicolour, and Iris Virginica plant)?
* What factors do you think limited the number of data instances per class?
* How do you think the data was collected? What implication would this have for real world deployment of a model for automatic detection of iris classes based on this dataset?
* How do you think it was labelled? What kind of challenge might this pose for collection of more training data (and labels) for automatic detection of iris classes?